# Sistema de Preguntas y Respuestas sobre Legislación Paraguaya

## Alumnos

1.   Oscar Ortíz
2.   Guillermo Osorio

In [ ]:
# -*- coding: utf-8 -*-
# Script para realizar consultas a una base de datos legal paraguaya usando recuperación aumentada por generación (RAG)

import os
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFaceHub
from langchain.schema import Document
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from dotenv import load_dotenv

# Cargar variables de entorno desde .env (incluye clave API de OpenAI)
load_dotenv()

# Cargar el índice FAISS previamente generado con embeddings de HuggingFace
vectorstore = FAISS.load_local(
    "vectorstore_faiss",
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"),
    allow_dangerous_deserialization=True
)
retriever = vectorstore.as_retriever()

# Definir el modelo de lenguaje de OpenAI (GPT-3.5 Turbo en este caso)
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

# Plantilla personalizada para generar respuestas jurídicas basadas en el contexto legal
plantilla = """Eres un asistente jurídico entrenado en legislación paraguaya. Usá exclusivamente la información de los artículos legales proporcionados para responder.

Pregunta: {question}

Artículos relevantes:
{context}

Respuesta:
"""

# Construcción del prompt
prompt = PromptTemplate(
    template=plantilla,
    input_variables=["question", "context"]
)

# Encadenar el modelo LLM con el prompt definido
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Crear una cadena para combinar documentos usando el LLM
qa_chain = StuffDocumentsChain(llm_chain=llm_chain, document_variable_name="context")

# Envolverlo todo en una cadena RetrievalQA
retrieval_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=qa_chain,
    return_source_documents=True
)

# Función para buscar y responder una pregunta
def buscar_y_responder(pregunta):
    resultado = retrieval_chain.invoke({"query": pregunta})
    respuesta = resultado["result"]
    fuentes = resultado.get("source_documents", [])

    if not respuesta.strip():
        respuesta = "No se encontró información relevante en la legislación disponible."

    print("\n📝 Respuesta:\n", respuesta)
    print("\n🔎 Fuente(s):")
    for doc in fuentes:
        meta = doc.metadata
        print(f" - Ley {meta.get('nro_ley')} - {meta.get('titulo_ley')} - Artículo {meta.get('nro_articulo')}")

# Demostración de uso
print("📚 Sistema de Consulta sobre Legislación Paraguaya")
print("Escribí tu pregunta o 'salir' para terminar.")

# Ejecución interactiva (simulado aquí con preguntas de ejemplo del archivo output.txt)
preguntas = [
    "¿Hasta qué edad una persona es considerada menor de edad según la legislación paraguaya y qué implicancias legales tiene?",
    "¿Cuáles son los derechos garantizados por la Constitución Nacional del Paraguay para los pueblos indígenas?",
    "¿Qué establece el Código del Trabajo paraguayo sobre la jornada máxima de trabajo y el pago por horas extras?",
    "¿Qué derechos tiene un trabajador despedido sin causa justificada en Paraguay y qué tipo de indemnización le corresponde?",
    "¿Qué requisitos se deben cumplir para contraer matrimonio legalmente en Paraguay según el Código Civil?",
    "¿Cómo se define la propiedad privada en la Constitución paraguaya y en qué casos puede ser expropiada?",
    "¿Qué normas establece la legislación paraguaya para la protección del ambiente y los recursos naturales?",
    "¿Cuáles son los pasos legales para adoptar un menor en Paraguay según el Código de la Niñez y la Adolescencia (si estuviera cargado)?",
    "¿Qué dice la Constitución Nacional del Paraguay sobre la libertad de prensa y de expresión?",
    "¿Qué establece el marco legal paraguayo respecto a la igualdad entre hombres y mujeres en el ámbito laboral?",
    "¿Puede el presidente del Paraguay ser reelegido?",
    "¿Qué derechos tengo como ciudadano en caso de ser detenido?",
    "¿Está garantizada la libertad de prensa en el país?",
    "¿Cuántos días de vacaciones me corresponden por ley?",
    "¿Pueden despedirme sin justificación legal?",
    "¿Cuál es el salario mínimo legal en Paraguay?",
    "¿Qué requisitos debo cumplir para casarme legalmente?",
    "¿Cómo se reparten los bienes en caso de divorcio?",
    "¿Puedo desheredar a un hijo?",
    "¿Qué pena se aplica por hurto?",
    "¿Cuándo un delito prescriba en Paraguay?",
    "¿Es delito grabar a alguien sin su consentimiento?",
    "¿A qué edad puedo votar en Paraguay?",
    "¿Puedo ser candidato si no tengo afiliación política?",
    "¿Qué pasa si no voto en una elección obligatoria?",
    "¿Es obligatorio usar casco en motocicleta?",
    "¿Puedo conducir si tengo 17 años?",
    "¿Qué sanciones existen por manejar alcoholizado?",
    "¿Qué derechos tengo si mi proveedor de internet falla constantemente?",
    "¿El Estado puede intervenir mis comunicaciones?",
    "¿Puedo portar una antena de radio sin autorización?",
    "¿A qué edad un niño puede trabajar legalmente?",
    "¿Qué protección tiene un menor contra la violencia familiar?",
    "¿Es legal que un adolescente sea juzgado como adulto?",
    "¿Dónde puede acudir una mujer víctima de violencia doméstica?",
    "¿Qué medidas puede tomar la ley para proteger a una mujer en riesgo?",
    "¿La ley contempla sanciones para el acoso laboral hacia mujeres?",
    "¿El transporte público debe estar adaptado para personas con discapacidad?",
    "¿Las instituciones deben ofrecer educación inclusiva?",
    "¿Qué apoyo puede recibir una persona con discapacidad para conseguir empleo?",
    "¿Puedo pedir que una empresa borre mis datos?",
    "¿Qué pasa si una empresa vende mis datos sin permiso?",
    "¿Tengo derecho a saber qué información tienen sobre mí?",
    "¿Es obligatorio que las universidades estén acreditadas?",
    "¿Qué derechos tienen los estudiantes universitarios?",
    "¿Puedo reclamar si me cobran una matrícula ilegal?",
    "¿Puedo pedir información al gobierno como ciudadano común?",
    "¿Hay límites a la información pública que puedo solicitar?",
    "¿Qué plazos tiene el Estado para responder a mis pedidos?",
    "¿Puedo devolver un producto defectuoso sin perder mi dinero?",
    "¿Qué pasa si una empresa no respeta los precios publicados?",
    "¿Qué derechos tengo si se corta un servicio contratado?",
    "¿Puedo vender una obra antigua que encontré en mi casa?",
    "¿Qué está prohibido hacer en un sitio considerado patrimonio cultural?",
    "¿El Estado puede intervenir en propiedades privadas por razones culturales?",
    "¿Qué requisitos tiene un extranjero para residir legalmente en Paraguay?",
    "¿Puede un extranjero ser deportado sin juicio?",
    "¿Hay derechos especiales para refugiados o asilados?",
    "salir"
]

for pregunta in preguntas:
    if pregunta.lower() in ["salir", "exit"]:
        break
    print(f"➤ Pregunta: >? {pregunta}")
    buscar_y_responder(pregunta)

hola
